# LFRic W1/W2 Vector field on the cubed-sphere
This notebook shows how to regrid a vector field defined from a source to a targert mesh. The components of the vector field are staggered according to either Arakawa C (W2) or D (W1). Schematically,

In [ ]:
# Arakawa C or LFRic W2 space
# o---v----o
# |        |
# u        u
# |        |
# o---v----o

# Arakawa D or LFRic W1 space
# o---u----o
# |        |
# v        v
# |        |
# o---u----o

Mimetic interpolation leverages different (vector) interpolating functions for W1 and W2 fields.

In [ ]:
import numpy
import mint
print(f'mint version = {mint.__version__}')

## Load the source and target grids from Ugrid files

Both the source and target meshes and data must comply to the Ugrid 2d, netcdf conventions (https://ugrid-conventions.github.io/ugrid-conventions/).

In [ ]:
import iris
from iris.experimental.ugrid import Connectivity, Mesh, PARSE_UGRID_ON_LOAD

In [ ]:
# read the src data
src_filename = '../../data/cs8_wind.nc'
with PARSE_UGRID_ON_LOAD.context():
    src_u_cube = iris.load_cube(src_filename, 'eastward_wind_at_cell_faces')
    src_v_cube = iris.load_cube(src_filename, 'northward_wind_at_cell_faces')

In [ ]:
# get the source mesh and its connectivity
src_x = src_u_cube.mesh.node_coords.node_x.points
src_y = src_u_cube.mesh.node_coords.node_y.points
# edge to node connectivity
src_e2n = src_u_cube.mesh.edge_node_connectivity.indices_by_location()
src_e2n -= src_u_cube.mesh.edge_node_connectivity.start_index
# face to node connectivity
src_f2n = src_u_cube.mesh.face_node_connectivity.indices_by_location()
src_f2n -= src_u_cube.mesh.face_node_connectivity.start_index

In [ ]:
# read the target data, mesh and connectivity
tgt_filename = '../../data/cs2_wind.nc'
with PARSE_UGRID_ON_LOAD.context():
    tgt_u_cube = iris.load_cube(tgt_filename, 'eastward_wind_at_cell_faces')
    tgt_v_cube = iris.load_cube(tgt_filename, 'northward_wind_at_cell_faces')
# get the target mesh and its connectivity
tgt_x = tgt_u_cube.mesh.node_coords.node_x.points
tgt_y = tgt_u_cube.mesh.node_coords.node_y.points
# edge to node connectivity
tgt_e2n = tgt_u_cube.mesh.edge_node_connectivity.indices_by_location()
tgt_e2n -= tgt_u_cube.mesh.edge_node_connectivity.start_index
# face to node connectivity
tgt_f2n = tgt_u_cube.mesh.face_node_connectivity.indices_by_location()
tgt_f2n -= tgt_u_cube.mesh.face_node_connectivity.start_index   

## Regrid the vector field

In [ ]:
# grid options for a cubed-sphere grid, use (0, 0, 1) for a rectilinear grid
src_flags = (1, 1, 1)
tgt_flags = (1, 1, 1)
# compute the regridding weights
rg = mint.IrisMintRegridder(src_u_cube.mesh, tgt_u_cube.mesh,
                            src_flags=src_flags, tgt_flags=tgt_flags)
# note: we assume the vector field to be Arakawa C (or W2)
result_u, result_v = rg.regrid_vector_cubes(src_u_cube, src_v_cube, fs=mint.FUNC_SPACE_W2)

In [ ]:
# apply the weights
regridder.vectorApply(src_u_cube.data, src_v_cube.data, tgt_u_cube.data, )

In [ ]:
def get_mid_edge_positions_from_vertices(x, y, e2n):
    """
    Function to return the mid edge positions, taking into account the periodicity 
    of the domain along longitudes
    :param x: array of longitudes (deg east)
    :param y: array of latitudes (deg north)
    :param e2n: edge to node connectivity, array of size (num_edges, 2)
    :returns two 1D arrays for (lon, lat)
    """
    i0, i1 = e2n[:, 0], e2n[:, 1]
    x0, x1 = x[i0], x[i1]
    y0, y1 = y[i0], y[i1]
    # add/subtract a periodicity length
    dx = x1 - x0
    x1 = numpy.where(dx > 180., x1 - 360., x1)
    x1 = numpy.where(dx < -180., x1 + 360., x1)
    # edge mid points
    xm = 0.5*(x0 + x1)
    ym = 0.5*(y0 + y1)
    return xm, ym

In [ ]:
from matplotlib.pyplot import quiver
src_xm, src_ym = get_mid_edge_positions_from_vertices(src_x, src_y, src_e2n)
quiver(src_xm, src_ym, src_u_cube.data, src_v_cube.data, color='g')
tgt_xm, tgt_ym = get_mid_edge_positions_from_vertices(tgt_x, tgt_y, tgt_e2n)
quiver(tgt_xm, tgt_ym, tgt_u_cube.data, tgt_v_cube.data, color='r')